In [ ]:
# --- Fiyat Karşılaştırma Programı (Colab İçin Tek Blok) ---

import pandas as pd
from google.colab import files

# 1) Kullanıcıdan Excel dosyasını yüklemesini iste
print("Lütfen fiyat dosyanızı (.xlsx) yükleyin.")
print("NOT: İlk sütun ürün adı, diğer sütunlar tedarikçi fiyatları olmalı.")
uploaded = files.upload()

# Yüklenen ilk dosyanın adını al
file_name = list(uploaded.keys())[0]
print(f"\nYüklenen dosya: {file_name}")

# 2) Excel dosyasını oku
df = pd.read_excel(file_name)

print("\n--- Ham Veri (İlk 5 Satır) ---")
print(df.head().to_string(index=False))

# 3) Tedarikçi sütunlarını belirle (ilk sütun ürün adı kabul ediliyor)
supplier_cols = df.columns[1:]

# 4) Her ürün için en ucuz fiyatı ve o fiyatı veren tedarikçiyi bul
df["En_Ucuz_Fiyat"] = df[supplier_cols].min(axis=1)
df["En_Ucuz_Tedarikçi"] = df[supplier_cols].idxmin(axis=1)

# 5) Her tedarikçi için "En ucuzdan fark" ve "Yüzde fark" sütunları oluştur
for col in supplier_cols:
    fark_col = f"{col}_Fark"
    yuzde_fark_col = f"{col}_Yuzde_Fark"

    df[fark_col] = df[col] - df["En_Ucuz_Fiyat"]
    df[yuzde_fark_col] = (df[fark_col] / df["En_Ucuz_Fiyat"]) * 100

# 6) Ürünleri en ucuz fiyata göre sıralayalım (ucuzdan pahalıya)
df_sirali = df.sort_values(by="En_Ucuz_Fiyat", ascending=True)

print("\n--- Karşılaştırma Sonucu (İlk 10 Satır) ---")
gosterilecek_sutunlar = ["En_Ucuz_Fiyat", "En_Ucuz_Tedarikçi"] + list(supplier_cols)
# Ürün sütunu varsa başa al
if df_sirali.columns[0] not in ["En_Ucuz_Fiyat", "En_Ucuz_Tedarikçi"]:
    gosterilecek_sutunlar = [df_sirali.columns[0]] + gosterilecek_sutunlar

print(df_sirali[gosterilecek_sutunlar].head(10).to_string(index=False))

# 7) Sonucu yeni bir Excel dosyasına kaydet
output_name = "fiyat_karsilastirma_sonuc.xlsx"
df_sirali.to_excel(output_name, index=False)

print(f"\nTüm karşılaştırma sonuçları '{output_name}' dosyasına kaydedildi.")
print("Dosyayı indirmek için aşağıdaki butonu kullanabilirsiniz.")

# 8) Dosyayı indirme
files.download(output_name)

In [ ]:
# --- Fiyat Karşılaştırma Programı (Otomatik Tedarikçi İsimli, Başlıktan Bağımsız) ---

import pandas as pd
from google.colab import files

print("Lütfen fiyat dosyasını (.xlsx) yükleyin.")
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

df = pd.read_excel(file_name)

print("\n--- Ham Veri (İlk 5 Satır) ---")
print(df.head().to_string(index=False))

# -----------------------------
# 1) Sütunları otomatik tanıma
# -----------------------------
tum_sutunlar = list(df.columns)
urun_sutunu = tum_sutunlar[0]
other_cols = tum_sutunlar[1:]  # tedarikçi çiftleri burada

pairs = []             # [(tedarikçi adı, fiyat sütunu, iskonto sütunu)]
i = 0

while i < len(other_cols):
    price_col = other_cols[i]
    discount_col = other_cols[i+1] if i+1 < len(other_cols) else None

    # Tedarikçi adı = fiyat sütun adının kendisi
    supplier_name = str(price_col).strip()

    pairs.append((supplier_name, price_col, discount_col))
    i += 2

print("\nTespit edilen tedarikçiler:", [p[0] for p in pairs])

# -----------------------------
# 2) Boş iskonto değerlerini 0 yap
# -----------------------------
discount_cols = [p[2] for p in pairs if p[2] is not None]
df[discount_cols] = df[discount_cols].fillna(0)

# -----------------------------
# 3) Net fiyat hesaplama
# -----------------------------
net_cols = []
for supplier_name, price_col, discount_col in pairs:
    net_col = f"{supplier_name}_Net"
    if discount_col is not None:
        df[net_col] = df[price_col] * (1 - df[discount_col] / 100)
    else:
        df[net_col] = df[price_col]
    net_cols.append(net_col)

print("\nOluşturulan net fiyat kolonları:", net_cols)

# -----------------------------
# 4) En ucuz tedarikçiyi bulma
# -----------------------------
df["En_Ucuz_Net_Fiyat"] = df[net_cols].min(axis=1)

# En ucuz net fiyatı veren tedarikçi sütununu bul
min_net_col = df[net_cols].idxmin(axis=1)

# Örn: "Hes_Net" → "Hes"
df["En_Ucuz_Tedarikçi"] = min_net_col.str.replace("_Net", "", regex=False)

# -----------------------------
# 5) Fark hesapları
# -----------------------------
for supplier_name, price_col, discount_col in pairs:
    net_col = f"{supplier_name}_Net"
    fark_col = f"{supplier_name}_Fark"
    yuzde_fark_col = f"{supplier_name}_Yuzde_Fark"

    df[fark_col] = df[net_col] - df["En_Ucuz_Net_Fiyat"]
    df[yuzde_fark_col] = (df[fark_col] / df["En_Ucuz_Net_Fiyat"]) * 100

# -----------------------------
# 6) Sırala
# -----------------------------
df_sirali = df.sort_values(by="En_Ucuz_Net_Fiyat", ascending=True)

print("\n--- Özet Sonuç (İlk 10 Satır) ---")
ozet_sutunlar = [urun_sutunu, "En_Ucuz_Tedarikçi", "En_Ucuz_Net_Fiyat"] + other_cols
print(df_sirali[ozet_sutunlar].head(10).to_string(index=False))

# -----------------------------
# 7) Excel'e kaydet
# -----------------------------
output_file = "fiyat_karsilastirma_sonuc_otomatik.xlsx"
df_sirali.to_excel(output_file, index=False)

print(f"\nSonuç dosyası oluşturuldu: {output_file}")
files.download(output_file)